# CS-GY 6313 / CUSP-GX 6006: Data Visualization - Spring '24
# Homework #4: Interactive Visualizations

In this homework assignment, you will be focusing on adding interactive components to 2D plots built with [`Altair`](https://altair-viz.github.io/gallery/index.html). This should give you an idea on how to generate interactivity in your plots.

## Interactive Weather Plots (15 points)

The code below is helper code to get you started. Let's import the necessary packages for Altair and the weather data that we'll be using in this assignment.

In [1]:
""" ===========================
=== DO NOT MODIFY THIS CODE ===
=========================== """

# Installling the necessary packages
!pip install "altair[all]"
import altair as alt
from vega_datasets import data

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


The next code block handles the data import for the Seattle weather data between 2012 to 2015. Meanwhile, we define the color scale that maps particular weather types to colors. We will use this color mapping when producing visualizations of each weather type.

In [18]:
""" ===========================
=== DO NOT MODIFY THIS CODE ===
=========================== """

# Import the seattle weather data
source = data.seattle_weather()

# Initialize some parameters for the data
scale = alt.Scale(domain=['sun', 'fog', 'drizzle', 'rain', 'snow'],
                  range=['#e7ba52', '#a7a7a7', '#aec7e8', '#1f77b4', '#9467bd'])
color = alt.Color('weather:N', scale=scale)

In [19]:
source

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
...,...,...,...,...,...,...
1456,2015-12-27,8.6,4.4,1.7,2.9,fog
1457,2015-12-28,1.5,5.0,1.7,1.3,fog
1458,2015-12-29,0.0,7.2,0.6,2.6,fog
1459,2015-12-30,0.0,5.6,-1.0,3.4,sun


### Part 1. Changing Plots (5 points)

Change the bottom chart such that, instead of showing the number of days that correspond to each weather type, the bottom chart is a horizontal bar chart that shows the amount of precipitation for each day. The bar charts must be a stacked bar chart where the x-axis represents the sum of the amount of precipitation that day, the y-axis represents the date (in month-date format), and the stacking must indicate the percentage each weather type contributed to the total sum. The stacking must also be color-coded to reflect the colors in the `scale` provided.



### Part 2: Weather Type Selection (5 points)

Modify the top graph so that if you click the weather name in the legend, all data points with different weather types are grayed out in the top chart. If I click anywhere else, the legend is reset and all colors are returned to the top chart.

### Part 3: Interval Selection (5 points)

Modify the top chart so that if you create a selectio of data points by clicking and dragging your mouse, you will see the bottom horizontal bar chart reflect the data points in the selection only.

## Your Code:

In [20]:
""" ==============================================
=== You can modify the code starting from here ===
============================================== """

points = alt.Chart().mark_point().encode(
    alt.X('temp_max:Q', title='Maximum Daily Temperature (C)'),
    alt.Y('temp_range:Q', title='Daily Temperature Range (C)'),
    size=alt.Size('precipitation:Q', scale=alt.Scale(range=[1, 200]))
).transform_calculate(
    "temp_range", "datum.temp_max - datum.temp_min"
).properties(
    width=600,
    height=400
)

bars = alt.Chart().mark_bar().encode(
    x='count()',
    y='weather:N',
    color=alt.Color('weather:N', scale=scale),
).transform_calculate(
    "temp_range", "datum.temp_max - datum.temp_min"
).properties(
    width=600
)

alt.vconcat(points, bars, data=source)

alt.VConcatChart(...)

# Part 1


In [27]:
points = alt.Chart().mark_point().encode(
    x=alt.X('monthdate(date):O', title='Date'),
    y=alt.Y('temp_max:Q', title='Maximum Daily Temperature (C)'),
    size=alt.Size('precipitation:Q', scale=alt.Scale(range=[1, 200])), 
    color=alt.Color('weather:N', scale=scale)
).properties(
    width=600,
    height=400
)


bars = alt.Chart().mark_bar().encode(
    x=alt.X('sum(precipitation):Q', title='Sum of  Precipitation (mm)'),
    y=alt.Y('monthdate(date):O', title='Date'),
    color=alt.Color('weather:N', scale=scale),
    order=alt.Order('weather:N', sort='ascending')  # To maintain consistent stack order
).transform_filter(
    alt.datum.precipitation > 0  # Assuming we filter out days with no precipitation
).properties(
    width=600, 
   height=400
)

combined_chart = alt.vconcat(points, bars, data=source)

combined_chart  # This will display the chart

alt.VConcatChart(...)

# Part 2

In [30]:
selection = alt.selection_point(fields=['weather'], bind='legend')


points = alt.Chart().mark_point().encode(
    x=alt.X('monthdate(date):O', title='Date'),
    y=alt.Y('temp_max:Q', title='Maximum Daily Temperature (C)'),
    size=alt.Size('precipitation:Q', scale=alt.Scale(range=[1, 200])), 
    color=alt.condition(selection, 'weather:N', alt.value('lightgray'))
).properties(
    width=600,
    height=400
).add_selection(selection)



# Combine the points and bars into a single vertical concatenated chart
combined_chart = alt.vconcat(points, bars, data=source)

combined_chart   # This will display the chart


/Users/juniorgarcia/opt/anaconda3/envs/vizhw1/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.VConcatChart(...)

# Part 3

In [35]:
interval_selection = alt.selection_interval(encodings=['x'], name='date_interval')
legend_selection = alt.selection_multi(fields=['weather'], bind='legend')

points = alt.Chart().mark_point().encode(
    x=alt.X('monthdate(date):O', title='Date'),
    y=alt.Y('temp_max:Q', title='Maximum Daily Temperature (C)'),
    size=alt.Size('precipitation:Q', scale=alt.Scale(range=[1, 200])), 
    color=alt.condition(interval_selection & legend_selection, 'weather:N', alt.value('lightgray'))
).properties(
    width=600,
    height=400
).add_selection(
    interval_selection
).add_selection(
    legend_selection
)

bars = alt.Chart().mark_bar().encode(
    x=alt.X('sum(precipitation):Q', title='Total Precipitation (mm)'),
    y=alt.Y('monthdate(date):O', title='Date'),
    color=alt.Color('weather:N', scale=scale),
    order=alt.Order('weather:N', sort='ascending')  # To maintain consistent stack order
).transform_filter(
    alt.datum.precipitation > 0  # Assuming we filter out days with no precipitation
).transform_filter(
    interval_selection
).transform_filter(
    legend_selection
).properties(
    width=600, 
    height=400
)




combined_chart = alt.vconcat(points, bars, data=source)

combined_chart  


/Users/juniorgarcia/opt/anaconda3/envs/vizhw1/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/juniorgarcia/opt/anaconda3/envs/vizhw1/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.VConcatChart(...)